In [1]:
class World:
    """
    Represents the simulation environment.

    The World handles:
    - Population tracking
    - Running each simulation tick
    - Aggregating violence and economics
    - Updating global utilities and triggering role transitions
    """

    def __init__(self, n_farmers, n_bandits, n_soilders):
        """
        Initialize world with given population counts.
        """
        self.people = []
        self.global_tick = 0

        for _ in range(n_farmers):
            self.people.append(Person("farmer", wealth=1.0))
        for _ in range(n_bandits):
            self.people.append(Person("bandit", wealth=0.7))
        for _ in range(n_soilders):
            self.people.append(Person("soldier", wealth=0.8))

        self.total_robberies = 0
        self.total_suppressions = 0
        self.total_violence_last_tick = 0.0

        self.utilities_for_roles = {"farmer": 0.0, "bandit": 0.0, "soldier": 0.0}

        self.update_population_counts()

        # Diagnostics
        self.expected_prod_per_farmer = 0.0
        self.expected_rob_per_bandit = 0.0
        self.expected_soldier_pay = 0.0

    def update_population_counts(self):
        """
        Recalculate number of farmers, bandits, and soldiers.
        Should be called after role switches.
        """
        self.farmer_pop = sum(p.is_farmer() for p in self.people)
        self.bandit_pop = sum(p.is_bandit() for p in self.people)
        self.soldier_pop = sum(p.is_soldier() for p in self.people)

    def step(self):
        """
        Advance the simulation by one tick.

        This function performs:
        1. Production
        2. Robbery attempts
        3. Suppression by soldiers
        4. Tax redistribution
        5. Psychology updates
        6. Forward utility calculation
        7. Role switching
        """
        self.global_tick += 1

        # Reset all agents for new cycle
        for p in self.people:
            p.reset_for_new_tick()

        farmers = [p for p in self.people if p.is_farmer()]
        bandits = [p for p in self.people if p.is_bandit()]
        soldiers = [p for p in self.people if p.is_soldier()]

        # --- Production phase ---
        for f in farmers:
            f.produce(params["farmer_productivity"])

        # --- Robbery phase ---
        self.total_robberies = 0
        random.shuffle(bandits)

        for b in bandits:
            if not farmers:
                break
            target = max(farmers, key=lambda x: x.wealth)

            _, success = b.attempt_rob(target, params["steal_rate"])
            if success:
                self.total_robberies += 1

        # --- Suppression phase ---
        self.total_suppressions = 0
        bandits = [p for p in self.people if p.is_bandit()]

        for s in soldiers:
            if not bandits:
                break

            target = random.choice(bandits)

            _, success = s.attempt_suppress(
                target_bandit=target,
                p_hit=params["suppression_prob"],
                confiscate_rate=params["confiscate_rate"],
            )

            if success:
                self.total_suppressions += 1

        # --- Taxation + soldier pay ---
        tax_pool = sum(f.pay_tax(params["tax_rate"]) for f in farmers)
        total_base = params["soldier_base_pay"] * len(soldiers)

        pay_per = (tax_pool + total_base) / len(soldiers) if soldiers else 0.0

        for s in soldiers:
            s.receive_pay(pay_per)

        # --- Diagnostics (expected values) ---
        self.expected_prod_per_farmer = (
            sum(f.last_production for f in farmers) / len(farmers) if farmers else 0.0
        )
        self.expected_rob_per_bandit = (
            sum(b.last_rob_gain for b in bandits) / len(bandits) if bandits else 0.0
        )
        self.expected_soldier_pay = (
            sum(s.last_pay for s in soldiers) / len(soldiers) if soldiers else 0.0
        )

        # --- Violence index ---
        current_violence = self.total_robberies + self.total_suppressions
        self.total_violence_last_tick = current_violence

        # --- Forward utilities ---
        n = len(self.people)

        farmer_u = sum(p.predicted_utility("farmer", self, params) for p in self.people) / n
        bandit_u = sum(p.predicted_utility("bandit", self, params) for p in self.people) / n
        soldier_u = sum(p.predicted_utility("soldier", self, params) for p in self.people) / n

        # Hard lower bounds to prevent collapse artifacts
        self.utilities_for_roles = {
            "farmer": max(1.5, max(farmer_u, self.expected_prod_per_farmer)),
            "bandit": max(1.5, max(bandit_u, self.expected_rob_per_bandit)),
            "soldier": max(1.5, max(soldier_u, self.expected_soldier_pay) - 0.5)
        }

        # --- Update internal psychology ---
        for p in self.people:
            p.update_psychology(
                global_violence=current_violence,
                alpha=params["alpha_perception"],
                fear_inc=params["fear_increment"],
                fear_dec=params["fear_decay"],
            )

        # --- Role switching ---
        for p in self.people:
            p.decide_switch(
                utilities=self.utilities_for_roles,
                switch_urge_threshold=params["switch_urge_threshold"],
                utility_margin=random.uniform(0.3, 2),
                global_tick=self.global_tick,
            )

        for p in self.people:
            p.finalize_role_switch()

        self.update_population_counts()

    def to_dict(self):
        """
        Export global state for visualization or CSV logging.
        """
        return {
            "farmer_pop": self.farmer_pop,
            "bandit_pop": self.bandit_pop,
            "soldier_pop": self.soldier_pop,
            "total_robberies": self.total_robberies,
            "total_suppressions": self.total_suppressions,
            "utilities": self.utilities_for_roles,
            "exp_prod_farmer": self.expected_prod_per_farmer,
            "exp_rob_bandit": self.expected_rob_per_bandit,
            "exp_pay_soldier": self.expected_soldier_pay,
        }
